# Playing the game

This notebook provides companion code for my video on The Kelly Criterion. The video and instructions below should be enough to play around. Enjoy! 

In [1]:
from betting_strategies import ConstantDollar, BetLikeADummy, Kelly, KellyWithAnEstimate
import altair as alt

alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

The classes `ConstantDollar`, `BetLikeADummy`, `Kelly` and `KellyWithAnEstimate` are all subclasses of `BaseGame` and provide different types of strategy. If you want to get fancy, you could implement your own classes by overriding the `_strategy` method. If you'd like a deep understanding, read the code - I've tried to comment it to make it understandable.

We can start by inspecting the arguments of the ConstantDollar game (all inits are the same)

In [2]:
help(ConstantDollar.__init__)

Help on function __init__ in module betting_strategies:

__init__(self, prob_heads: float, payout_ratio: float, N_flips: int, N_games: int, initial_wealth: float = 50)
    Initialization for all BaseGame-s - don't override it! The game is determined by the probability of heads,
    the payout ratio of payout-to-wager for a bet on heads and the number of flips.
    
    Note : Without lose of generaltiy, we assume every wager is on heads. This is because the game is such that it's
    always best to bet on one outcome every time.
    
    Parameters
    ----------
    prob_heads : true probability of heads on each flip
    payout_ratio : if you wager wager-dollars on a flip for heads, you'll receive wager*payout_ratio, otherwise
    you'll lose wager.
    N_flips : The number of flips involved in one play of the game.
    N_games : The number of games you'll play to test your strategy
    initial_wealth : The amount of wealth you begin each game with.



In [3]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [4]:
prob_heads = 0.7
payout_ratio = 4 / 6
N_flips = 100
N_games = 1000  # Careful - if this number is too high, things can get slow
initial_wealth = 50

args = dict(
    prob_heads=prob_heads,
    payout_ratio=payout_ratio,
    N_flips=N_flips,
    N_games=N_games,
    initial_wealth=initial_wealth,
)

constant_dollar = ConstantDollar(**args)
dummy = BetLikeADummy(**args)
kelly = Kelly(**args)
kelly_w_estimate = KellyWithAnEstimate(**args)

<IPython.core.display.Javascript object>

### Constant Dollar Wager

In [5]:
constant_dollar.amount = (
    5  # You can use this to change the constant dollar wage - here it's 5
)
constant_dollar.plot_games(n_games=25, log=False, opacity=0.5)

alt.Chart(...)

<IPython.core.display.Javascript object>

In [6]:
constant_dollar.plot_growth_rate_distribution(
    n_games=N_games, min_max_growth_rate=[-0.04, 0.04], step_size=0.002
)

alt.Chart(...)

<IPython.core.display.Javascript object>

# Betting big when seeing a string of tails!

In [7]:
dummy.plot_games(n_games=50, log=False, opacity=0.5)

alt.Chart(...)

<IPython.core.display.Javascript object>

In [8]:
dummy.plot_growth_rate_distribution(
    n_games=N_games, min_max_growth_rate=[-0.04, 0.04], step_size=0.002
)

alt.Chart(...)

<IPython.core.display.Javascript object>

# Kelly Betting

In [9]:
kelly.plot_games(n_games=25, log=True, opacity=0.5)

alt.Chart(...)

<IPython.core.display.Javascript object>

In [10]:
kelly.plot_growth_rate_distribution(
    n_games=N_games, min_max_growth_rate=[-0.1, 0.1], step_size=0.002
)

alt.Chart(...)

<IPython.core.display.Javascript object>

In [11]:
kelly.plot_exp_growth_rates_by_perc_wager()

alt.LayerChart(...)

<IPython.core.display.Javascript object>

# Kelly Betting with an Estimate

In [12]:
kelly_w_estimate.plot_games(n_games=25, log=True, opacity=0.5)

alt.Chart(...)

<IPython.core.display.Javascript object>

In [13]:
kelly_w_estimate.plot_growth_rate_distribution(
    n_games=N_games, min_max_growth_rate=[-0.1, 0.1], step_size=0.002
)

alt.Chart(...)

<IPython.core.display.Javascript object>